# Phase V: INFO 2950 Final Project

## The Impact of Remote Work on AI Job Market Salary Hierarchies

**Author:** Angel Gomez (ag2483)


## Introduction And Relevance

As the AI job market continues to grow, it is particularly interesting to analyze the individuals driving this change and the salaries they are given. The rapid expansion of remote work has transformed how AI professionals work, raising questions about whether traditional salary hierarchies based on experience still hold in remote settings, and whether specialized technical roles command different premiums than generalist positions.

This research aims to identify patterns in salary differences based on experience level, role type, and work arrangement. For recent graduates and career changers, understanding how remote work affects salary progression informs critical decisions about career paths and negotiation strategies. For employers, these insights are essential for structuring competitive compensation in an evolving market where remote work and AI specialization reshape traditional pay structures.


***

### Research Question: 

In the US AI job market, does remote work reduce the salary premium traditionally associated with higher experience levels, and do specialized technical roles (Machine Learning Engineers, Research Scientists) show different patterns compared to generalist roles (Data Scientists, Data Analysts)?

**Important Definitions:**

- **Salary premium:** The percentage increase in compensation as workers advance from entry-level to senior-level positions, calculated as (Senior median salary - Entry median salary) / Entry median salary × 100%
- **Specialized roles:** Positions requiring deep machine learning and AI expertise (e.g., Machine Learning Engineer, Research Scientist, Computer Vision Engineer)
- **Generalist roles:** Broader data science and analytics positions (e.g., Data Scientist, Data Analyst, Data Engineer)

***

**Preview of Findings:**

## Data Description and Cleaning 

### Data Description:

**Overview**: 

This analysis uses surveyed salary data from AI and data science jobs postings from the following sites from 2020 - 2025: 

- aijobs.net Salary Dataset (CC0)
- 365datascience.com
- Payscale
- KDnuggets
- ZipRecruiter
- Wellfound (AngelList)

The raw dataset contains 151,445 global job postings across various AI-related roles and was puiled from Kaggle:

Kaggle Link: https://www.kaggle.com/datasets/adilshamim8/salaries-for-data-science-jobs

**Why was this dataset created?**

This dataset was created to provide insight into and understand global AI job market salary trends and employment patterns. This kind of analysis can help influence people's decision-making processes in the job market.


**Who created/funded this dataset?**

This dataset was independently funded by Adil Shamin and published independently on Kaggle in 2025.

**What do the instances repersent and how many are there??**

Each row represents a single job posting or reported position in the AI/data science field. There are a total of 151,445 instances in the raw dataset and 133,972 in the cleaned dataset.


**What are the attributes?**

- `work_year`: Year the position was posted (2025)
- `experience_level`: Entry, Mid, Senior, or Executive
- `job_title`: Specific role title (e.g., "Machine Learning Engineer")
- `role_type`: Categorized as "Specialized" (ML-focused) or "Generalist" (broader data roles) - *created during cleaning*
- `salary_in_usd`: Annual salary in USD
- `remote_ratio`: Percentage of work that is remote (0%, 50%, or 100%)
- `remote_work`: Binary indicator (0 = on-site, 1 = fully remote) - *created during cleaning*
- `company_size`: Small (S), Medium (M), or Large (L)

**Original Raw Columns:**

work_year, job_title, salary, salary_currency, salary_in_usd, experience_level, employment_type, company_location, company_size, employee_residence, remote_ratio


**What might influence what was observed?**

This dataset was created in 2025, so it captures only a single snapshot without historical context. The data was sourced from specific job boards (ai-jobs.net, ZipRecruiter, Payscale, etc.), meaning jobs posted on other platforms aren't included. The dataset also likely overrepresents major tech hubs where these platforms are more popular. Additionally, posted salary ranges may differ from actual negotiated offers. However, given the large dataset size (151,445 entries), these individual quirks likely have minimal impact on overall patterns.

**Were people notified/did they consent?**

No data collection was done in-person. All data collection was scraped from online job boards. Users who uploaded job postings and shared their information were not directly aware of this specific data collection effort, though the information was publicly posted.


***

### Cleaning

Data cleaning was completed in a seperate and independent notebook 

**Cleaning notebook:** 

https://github.com/ag2483-Angel/INFO2950FinalProject/blob/e0c771f5753df01c8edee05bd276e8d7fa59c1b7/data_cleaning.ipynb

**Explanation of Cleaning**
- Checked for duplicates
- Filtered to US employees only
- Filtered to full-time positions only
- Removed hybrid workers (kept only 0% or 100% remote)
- Created binary `remote_work` variable
- Categorized job titles into Specialized vs Generalist roles
- Mapped experience level codes to readable names
- Removed salary outliers using IQR method
- Selected relevant columns for analysis


I will bring in our important imports and cleaned datatset.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

df = pd.read_csv('cleaned_salaries.csv')

print(f"Dataset loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Dataset loaded: 131,407 rows, 8 columns

Columns: ['work_year', 'experience_level', 'job_title', 'role_type', 'salary_in_usd', 'remote_ratio', 'remote_work', 'company_size']


,work_year,experience_level,job_title,role_type,salary_in_usd,remote_ratio,remote_work,company_size
0,2025,Executive,Head of Data,Generalist,232344,0,0,M
1,2025,Senior,Data Scientist,Generalist,145400,0,0,M
2,2025,Senior,Data Scientist,Generalist,81600,0,0,M
3,2025,Mid,Engineer,Generalist,160000,100,1,M
4,2025,Mid,Engineer,Generalist,140000,100,1,M


### Data Limitations and Strengths

**Limitations**
Posted salaries may not reflect actual negotiated compensation, which could overstate or understate true market rates. The dataset lacks information on total compensation (equity, bonuses, benefits), meaning our salary comparisons may be incomplete—particularly for senior roles where equity often comprises a significant portion of pay. Without company identifiers, we cannot control for company-specific effects, so observed patterns might reflect which companies hire remote workers rather than remote work's true effect on salaries.

The class imbalance (79% on-site vs 21% remote) may reduce statistical power to detect remote work effects, and any significant p-values should be interpreted cautiously given this imbalance. Additionally, as a single-year snapshot, we cannot determine whether observed patterns are stable trends or temporary market conditions.

**Addressing Limitations:**

To address class imbalance, we include confidence intervals and focus on effect sizes rather than p-values alone. We control for company size in our regression models to partially account for company-level variation. We use median salaries rather than means for descriptive comparisons, which are more robust to the skewed salary distributions common when equity and bonuses are excluded. We also clearly frame our findings as correlational, not causal, and acknowledge in our conclusions what this data can and cannot tell us. Future research could address remaining limitations by incorporating longitudinal data, total compensation figures, and company-level controls.

**Strengths**

Large sample size (133,972 positions), multiple job board sources for broader coverage, standardized USD salary conversions, and recent 2025 data reflecting current market conditions.


### Preregistration Statements 

We preregistered two hypotheses and their analysis plans. This prevents p-hacking and ensures we report findings regardless of statistical significance.

## Preregistration #1: Remote Work and Salary Hierarchies

**Hypothesis:** Remote work reduces the salary premium associated with higher experience levels in the US AI job market. Specifically, the percentage increase in salary from entry-level to senior-level positions will be smaller for fully remote workers compared to on-site workers.

**Null Hypothesis (H₀):** The interaction between experience level and remote work is zero or positive—remote work does not flatten salary hierarchies.

**Alternative Hypothesis (H₁):** The interaction between experience level and remote work is negative—remote workers see smaller salary increases as they gain experience compared to on-site workers.

**Analysis:** Linear regression with salary_in_usd as the dependent variable and experience_level, remote_work, company_size, and the interaction term (experience_level × remote_work) as independent variables. If the interaction coefficients are negative and statistically significant (p < 0.05), this supports the hypothesis that remote work flattens salary hierarchies.

**Addressing class imbalance:** Given the 79% on-site vs 21% remote split, we will report 95% confidence intervals alongside p-values and focus on effect sizes to assess practical significance.

## Preregistration #2: Specialized vs Generalist Role Trajectories

**Hypothesis:** Specialized technical roles (Machine Learning Engineers, Research Scientists) show a steeper salary trajectory across experience levels compared to generalist roles (Data Scientists, Data Analysts), and this difference is more pronounced for on-site workers than remote workers.

**Null Hypothesis (H₀):** There is no significant difference in salary trajectories between specialized and generalist roles.

**Alternative Hypothesis (H₁):** Specialized roles have significantly steeper salary trajectories than generalist roles.

**Analysis:** 
- *Descriptive:* Calculate experience premium ((Senior Median - Entry Median) / Entry Median × 100%) for each of four groups: Specialized On-site, Specialized Remote, Generalist On-site, Generalist Remote.
- *Statistical Test:* Linear regression with salary_in_usd as the dependent variable and role_type, experience_level, remote_work, company_size, and the interaction term (role_type × experience_level) as independent variables. If the interaction coefficient is positive and significant (p < 0.05), specialized roles have steeper trajectories.

**Why this matters:** Understanding whether specialized training pays off differently across work settings informs career planning, educational investment decisions, and employer compensation strategies.

### Exploratory Data Analysis:

I will perform the analyses specified in the preregistration statements. 

**Analysis 1: Remote Work and Salary Hiearchies**

We first examine whether remote work flattens the salary premium associated with experience.


In [5]:
# Calculate experience premiums (Entry to Senior) for on-site vs remote
def calc_premium(data, remote_val):
    subset = data[data['remote_work'] == remote_val]
    entry = subset[subset['experience_level'] == 'Entry']['salary_in_usd'].median()
    senior = subset[subset['experience_level'] == 'Senior']['salary_in_usd'].median()
    premium = (senior - entry) / entry * 100
    return entry, senior, premium

onsite_entry, onsite_senior, onsite_premium = calc_premium(df, 0)
remote_entry, remote_senior, remote_premium = calc_premium(df, 1)

print("Experience Premium (Entry → Senior):")
print(f"  On-site: ${onsite_entry:,.0f} → ${onsite_senior:,.0f} = {onsite_premium:.1f}% increase")
print(f"  Remote:  ${remote_entry:,.0f} → ${remote_senior:,.0f} = {remote_premium:.1f}% increase")
print(f"\nDifference: {onsite_premium - remote_premium:.1f} percentage points")

Experience Premium (Entry → Senior):
  On-site: $95,680 → $167,200 = 74.7% increase
  Remote:  $87,500 → $160,000 = 82.9% increase

Difference: -8.1 percentage points
